In [7]:
from gurobipy import GRB, Model, quicksum
import pandas as pd

## Modelo

In [8]:
model = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-09


## Datos

In [9]:
parametros = pd.read_csv('data/parametros.csv', delimiter=';')
parametros_t = pd.read_csv('data/parametros_t.csv', index_col=0, delimiter=';')
parametros_i = pd.read_csv('data/parametros_i.csv', index_col=0, delimiter=';')
parametros_qit = pd.read_csv('data/parametros_qit.csv', index_col=0, delimiter=';')
parametros_pit = pd.read_csv('data/parametros_pit.csv', index_col=0, delimiter=';')

## Parámetros

In [10]:
c = parametros['c'][0]
b = parametros['b'][0]

In [11]:
d_t = parametros_t['dt'].tolist()
l_t = parametros_t['lt'].tolist()

In [12]:
Smax_i = parametros_i['Smax_i'].tolist()
s_i = parametros_i['s_i'].tolist()
e_i = parametros_i['e_i'].tolist()
f_i = parametros_i['f_i'].tolist()
g_i = parametros_i['g_i'].tolist()
a_i = parametros_i['a_i'].tolist()

In [13]:
q_i_t = parametros_qit.values.tolist()
p_i_t = parametros_pit.values.tolist()

## Variables

In [15]:
indices_i = [i for i in range(1, 5)]
indices_t = [i for i in range(0, 21)]
indices_n = [i for i in range(1, 1000)]

In [16]:
X = {}
Y = {}
I = {}
Z = {}
for i in indices_i:
    for t in indices_t:
        
        X[(i, t)] = model.addVar(vtype=GRB.INTEGER, name=f"x_{i}_{t}")
        Y[(i, t)] = model.addVar(vtype=GRB.INTEGER, name=f"y_{i}_{t}")
    
        for n in indices_n:
            I[(i, t, n)] = model.addVar(vtype=GRB.BINARY, name=f"I_{i}_{t}_{n}")
            Z[(i, t, n)] = model.addVar(vtype=GRB.BINARY, name=f"z_{i}_{t}_{n}")

model.update()

## Restricciones

In [17]:
# R1
model.addConstrs((X[(i, t)] >= Y[(i, t)] for i in indices_i for t in indices_t), name="R1");

In [18]:
# R2
model.addConstrs((quicksum(I[(i, t, n)] for n in indices_n) == Y[(i, t)] for i in indices_i for t in indices_t), name="R2");

In [19]:
# R3
model.addConstrs((Z[(i, t, n)] + I[(i, t-1, n)] >= I[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.1");
model.addConstrs((Z[(i, t, n)] + I[(i, t-1, n)] <= 2 - I[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.2");
model.addConstrs((I[(i, t, n)] <= Z[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.3");

In [20]:
# R4
model.addConstrs((Y[(i, 0)] == g_i[i-1] for i in indices_i), name="R4.1");
model.addConstrs((X[(i, 0)] == g_i[i-1] for i in indices_i), name="R4.2");

In [21]:
# R5
model.addConstrs((X[(1, t)] <= g_i[1-1] for t in indices_t), name="R5.1");
model.addConstrs((X[(i, t)] >= g_i[i-1] for i in indices_i[2:] for t in indices_t), name="R5.2");

In [22]:
# R6
model.addConstrs((X[(1, t)] >= X[(1, t+1)] for t in indices_t[0:20]), name="R6.1");
model.addConstrs((X[(i, t)] <= X[(i, t+1)] for i in indices_i[2:] for t in indices_t[:20]), name="R6.2");

In [23]:
# R7
model.addConstrs((quicksum(Y[(i, t)] for i in indices_i) >= d_t[t-1] for t in indices_t[1:]), name="R7");

In [24]:
# R8
model.addConstrs((Y[(1, t)] * c <= l_t[1-1] for t in indices_t[1:]), name="R8");

In [25]:
# R9
model.addConstrs((X[(i, t)] * s_i[i-1] <= Smax_i[i-1] for t in indices_t for i in indices_i), name="R9");

In [26]:
# R10
model.addConstrs(((X[(1, t)] - Y[(1, t)])*q_i_t[1-1][t-1] + (X[(2, t)] - Y[(2, t)])*q_i_t[2-1][t-1] >= 0.5 * d_t[t-1] for t in indices_t[1:]), name="R10");

In [27]:
def Pt(t):
    pt1 = (X[1, t - 1] - X[1, t]) * b
    pt2 = quicksum((X[i, t] - X[1, t - 1]) * e_i[i-1] for i in indices_i[2:5])
    pt3 = quicksum(Y[i, t] * p_i_t[i-1][t-1] for i in indices_i)
    return pt1 + pt2 + pt3

In [28]:
# R11
model.addConstrs((Pt(t_1) <= 3 * Pt(t_2) for t_1 in indices_t[1:] for t_2 in indices_t[1:]), name="R11");

In [29]:
# R12
model.addConstrs((quicksum(I[(i, t, n)] for t in indices_t[tp:tp + 9]) <= 7 for i in indices_i for n in indices_n for tp in indices_t[1:13]), name="R12");

In [30]:
# R13
model.addConstrs((X[(i, t)] >= 0 for i in indices_i for t in indices_t), name="R13.1");
model.addConstrs((Y[(i, t)] >= 0 for i in indices_i for t in indices_t), name="R13.2");

## Función objetivo

In [31]:
Costos_Fijos = (g_i[1-1] - X[(1, 20)]) * b + quicksum((X[(i, 20)] - g_i[i-1])*e_i[i-1] for i in indices_i[2:]) + quicksum(X[(i, t)] * f_i[i-1] for i in indices_i for t in indices_t[1:])
Costos_Variables = quicksum(Y[(i, t)] * p_i_t[i-1][t-1] * q_i_t[i-1][t-1] for i in indices_i for t in indices_t[1:]) + quicksum(Z[(i, t, n)] * a_i[i-1] for i in indices_i for t in indices_t[1:] for n in indices_n)

In [32]:
model.setObjective(Costos_Fijos + Costos_Variables, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 288723 rows, 168000 columns and 1161761 nonzeros
Model fingerprint: 0xed4ef01c
Variable types: 0 continuous, 168000 integer (167832 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 170 rows and 3996 columns
Presolve time: 0.08s

Explored 0 nodes (0 simplex iterations) in 0.33 seconds (0.21 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


In [33]:
# obj_val = model.objVal
# print(f"Objetivo: {obj_val}")